<a href="https://colab.research.google.com/github/FurkyT/Simple_AI_Chatbot/blob/main/Simple_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tflearn

**Gerekli Kütüphaneler**

In [ ]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import tflearn
import tensorflow as tf
import random
import json
import pickle

**Chat Verisi**

In [ ]:
data = {"intents": [
        {"tag": "greeting",
         "patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day", "Whats up"],
         "responses": ["Hello!", "Good to see you again!", "Hi there, how can I help?"],
         "context_set": ""
        },
        {"tag": "goodbye",
         "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day"],
         "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],
         "context_set": ""
        },
        {"tag": "age",
         "patterns": ["how old",  "what is your age", "how old are you", "age?"],
         "responses": ["I'm just created", "Maybe a minute maybe couple of hours old"],
         "context_set": ""
        },
        {"tag": "name",
         "patterns": ["what should I call you", "whats your name?","who are you"],
         "responses": ["You can call me Bot.", "I'm Bot", "I'm the Bot aka your helper"],
         "context_set": ""
        },
        {"tag": "cars",
         "patterns": ["do you know anything about cars", "are you interested in cars", "can I ask u something about cars"],
         "responses": ["I am interested in SUVs.", "My knowledge is kind of limited with Audi, Ford and BMW SUVs",],
         "context_set": ""
        },
        {"tag": "job",
         "patterns": ["what can you do", "what is your job", "what is your role in here"],
         "responses": ["I can help you to sell your car","I can give a estimated price for your used car"],
         "context_set": ""
        },
        {"tag": "sale",
         "patterns": ["I want to sell my car", "I wonder my car's price", "how much is my car worth"],
         "responses": ["I can calculate your car price in a second, just need some informations ","I need you to give me some informations to calculate your car price."],
         "context_set": ""
        }
   ]
}

**Preprocessing ve Model Tanımlama**

In [ ]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    wrds = nltk.word_tokenize(pattern)
    words.extend(wrds)
    docs_x.append(wrds)
    docs_y.append(intent["tag"])


  if intent["tag"] not in labels:
    labels.append(intent["tag"])  

words = [stemmer.stem(w.lower()) for w in words if  w not in "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x,doc in enumerate(docs_x):
  bag = []
  wrds = [stemmer.stem(w) for w in doc]

  for w in words:
    if w in wrds:
      bag.append(1)
    else:
      bag.append(0)

  output_row = out_empty[:]
  output_row[labels.index(docs_y[x])] = 1

  training.append(bag)
  output.append(output_row)

training = np.array(training)
output = np.array(output)

tf.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)




**Training**

In [ ]:
model.fit(training, output, n_epoch = 1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

**Bag of Words ü Ve Chat Fonksiyonunu Oluşturma**

In [ ]:
def bag_of_words(s, words):
  bag = [0 for _ in range(len(words))]

  s_words = nltk.word_tokenize(s)
  s_words = [stemmer.stem(word.lower()) for word in s_words]

  for se in s_words:
    for i, w in enumerate(words):
      if w == se:
        bag[i] = 1

  return np.array(bag)

def chat():
  print("Start talking with the bot!, Type quit when you want to quit")
  while True:
    inp = input("You: ")
    if inp.lower() == "quit":
      break

    results = model.predict([bag_of_words(inp, words)])
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data["intents"]:
      if tg['tag'] == 'sale' == tag:
        print("araç fiyat tahmin")
        print(tag)
      
      elif tg['tag'] == tag:
        responses = tg['responses']
        print(random.choice(responses))
        print(tag)

    
    

    




**MAİN**

In [ ]:
chat()